In [1]:
import pandas as pd
import os
import numpy as np
import ast
import pysam
import collections
from tqdm import tqdm
from Bio.Seq import Seq

In [6]:
#completeRunsSpecies=['PonPyg', 'PonAbe','GorGor',''hs1','PanTro','SymSyn'] - COMPLETED
#completeRunsSpecies=['PanPan']

#columnList=['Unnamed: 0.1','Unnamed: 0','Focus_Species','Loci','Query_Species','Matches','Hit_Similarity']
#directory = '/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/splitMatches/'
#outDirectory='/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/'
#for species in completeRunsSpecies:
#    fileLines=[]
#    for file in os.listdir('/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/splitMatches/'):
#        if species in str(file) and 'Filled.csv' in str(file):
#            df = pd.read_csv(directory+file)
#            #print(len(df))
#            for row in df.index:
#                fileLines.append([x for x in df.loc[row]])
#        else:
#            continue
#    newDF = pd.DataFrame(data=fileLines, columns=columnList).set_index("Unnamed: 0.1")
#    newDF.to_csv(outDirectory+str(species)+"_Matches_Filled_Combined.csv")

In [7]:
genomeNames={
'GorGor-mat':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028885495.2_NHGRI_mGorGor1-v2.0_mat_genomic.fna',
    'GorGor-pat':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028885475.2_NHGRI_mGorGor1-v2.0_pat_genomic.fna',
    'PanPan-mat':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028858845.2_NHGRI_mPanPan1-v2.0_mat_genomic.fna',
    'PanPan-pat':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028858825.2_NHGRI_mPanPan1-v2.0_pat_genomic.fna',
    'PanTro-pri':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028858775.2_NHGRI_mPanTro3-v2.0_pri_genomic.fna',
    'PanTro-alt':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028858805.2_NHGRI_mPanTro3-v2.0_alt_genomic.fna',
    'PonAbe-pri':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028885655.2_NHGRI_mPonAbe1-v2.0_pri_genomic.fna',
    'PonAbe-alt':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028885685.2_NHGRI_mPonAbe1-v2.0_alt_genomic.fna',
    'PonPyg-alt':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028885525.2_NHGRI_mPonPyg2-v2.0_alt_genomic.fna',
    'PonPyg-pri':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028885625.2_NHGRI_mPonPyg2-v2.0_pri_genomic.fna',
    'SymSyn-alt':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028878085.2_NHGRI_mSymSyn1-v2.0_alt_genomic.fna',
    'SymSyn-pri':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028878055.2_NHGRI_mSymSyn1-v2.0_pri_genomic.fna',
    'hs1':'/home/loftus/genomes/chm13v2/chm13v2.fasta'
}

In [8]:
T2TDict={'NC_060925.1':'chr1',
'NC_060926.1':'chr2',
'NC_060927.1':'chr3',
'NC_060928.1':'chr4',
'NC_060929.1':'chr5',
'NC_060930.1':'chr6',
'NC_060931.1':'chr7',
'NC_060932.1':'chr8',
'NC_060933.1':'chr9',
'NC_060934.1':'chr10',
'NC_060935.1':'chr11',
'NC_060936.1':'chr12',
'NC_060937.1':'chr13',
'NC_060938.1':'chr14',
'NC_060939.1':'chr15',
'NC_060940.1':'chr16',
'NC_060941.1':'chr17',
'NC_060942.1':'chr18',
'NC_060943.1':'chr19',
'NC_060944.1':'chr20',
'NC_060945.1':'chr21',
'NC_060946.1':'chr22',
'NC_060947.1':'chrX',
'NC_060948.1':'chrY'}
T2TDict2 = {y:x for x,y in T2TDict.items()}

In [9]:
directory = '/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/old/'
blatdirectory = '/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/'
outdirectory = '/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/'


myCutOffPercentage=.5

for speciesDataframe in os.listdir(directory):
    
    if '_6nt.csv' in speciesDataframe:
        
        #Current Species Name
        speciesName = str(speciesDataframe.split("_")[0])
        
        filteredDF2 = pd.read_csv(directory+speciesDataframe).set_index("ID")
            
            ########################################################################################################################
        # READ IN THE BLAT SIMILARITIES
        simDF = pd.read_csv(blatdirectory+str(speciesName)+'_Matches_Filled_06-12-2024.csv').replace("hs1.txt", "hs1")
        totalGoodHits=[]
        bestHit=[]
        bestHitValue=[]
        simDF['Best_Hit']='NONE'
        simDF['Best_Hit_Value']=100.0
        simDF['Total_Good_Hits']='NONE'
        for row in simDF.index:
            matches = ast.literal_eval(str(simDF.at[row,'Matches']))
            similarities = ast.literal_eval(str(simDF.at[row,'Hit_Similarity']))
            tempDict={}
            for x,y in zip(matches,similarities):
                tempDict[x]=float(y)

            maxKey = min(tempDict, key=tempDict.get)
            simDF.at[row, 'Best_Hit'] = maxKey
            simDF.at[row, 'Best_Hit_Value'] = tempDict[maxKey]
            simDF.at[row, 'Total_Good_Hits'] = {x:y for x,y in tempDict.items() if y<=myCutOffPercentage}
     
        for key in genomeNames.keys():

            filteredDF2[key+'_BestHit']='NONE'
            filteredDF2[key+'_BestHitValue']='NONE'
            filteredDF2[key+'_Total_LowDistance_Hits']='NONE'
            
            
        for row in simDF.index:
            loci = str(simDF.at[row,'Loci'])
            query = str(simDF.at[row,'Query_Species'])
            if loci in filteredDF2.index:
                filteredDF2.at[loci,query+"_BestHit"]= simDF.at[row,'Best_Hit']
                filteredDF2.at[loci,query+"_BestHitValue"]= simDF.at[row,'Best_Hit_Value']
                filteredDF2.at[loci,query+"_Total_LowDistance_Hits"]= simDF.at[row,'Total_Good_Hits']
            else:
                continue
                
        myMatchList=[]
        columnList=[key+'_BestHitValue' for key in genomeNames.keys()]

        for row in filteredDF2.index:

            tempList=[]
            for column in columnList:

                if filteredDF2.at[row,column]=='NONE':
                    continue

                elif float(filteredDF2.at[row,column])<=myCutOffPercentage:
                    tempList.append(column.split("_")[0])

                else:
                    continue

            if len(tempList)==0:
                myMatchList.append("TRUE_INSERTION")
            else:
                myMatchList.append('_'.join(sorted(tempList)))

        filteredDF2['All_Matches']=myMatchList
        print(len(filteredDF2['All_Matches']))
        filteredDF2.to_csv(outdirectory+str(speciesName)+'_50Distance_06-18-2024.csv')
    else:
        continue

101697
88956
85912
38552
41130
246367
227818
